# Polarizability using automatic differentiation

Simple example for computing properties using (forward-mode)
automatic differentiation.
For a more classical approach and more details about computing polarizabilities,
see Polarizability by linear response.

In [1]:
using DFTK
using LinearAlgebra
using ForwardDiff

# Construct PlaneWaveBasis given a particular electric field strength
# Again we take the example of a Helium atom.
function make_basis(ε::T; a=10., Ecut=30) where {T}
    lattice=T(a) * I(3)  # lattice is a cube of $a$ Bohrs
    # Helium at the center of the box
    atoms     = [ElementPsp(:He, psp=load_psp("hgh/lda/He-q2"))]
    positions = [[1/2, 1/2, 1/2]]

    model = model_DFT(lattice, atoms, positions, [:lda_x, :lda_c_vwn];
                      extra_terms=[ExternalFromReal(r -> -ε * (r[1] - a/2))],
                      symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid=[1, 1, 1])  # No k-point sampling on isolated system
end

# dipole moment of a given density (assuming the current geometry)
function dipole(basis, ρ)
    @assert isdiag(basis.model.lattice)
    a  = basis.model.lattice[1, 1]
    rr = [a * (r[1] - 1/2) for r in r_vectors(basis)]
    sum(rr .* ρ) * basis.dvol
end

# Function to compute the dipole for a given field strength
function compute_dipole(ε; tol=1e-8, kwargs...)
    scfres = self_consistent_field(make_basis(ε; kwargs...); tol)
    dipole(scfres.basis, scfres.ρ)
end;

With this in place we can compute the polarizability from finite differences
(just like in the previous example):

In [2]:
polarizability_fd = let
    ε = 0.01
    (compute_dipole(ε) - compute_dipole(0.0)) / ε
end

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770923774427                   -0.52    9.0         
  2   -2.772147774115       -2.91       -1.32    1.0    105ms
  3   -2.772170358956       -4.65       -2.50    1.0    119ms
  4   -2.772170650575       -6.54       -3.22    1.0    106ms
  5   -2.772170722152       -7.15       -3.97    2.0    123ms
  6   -2.772170722664       -9.29       -4.24    1.0    124ms
  7   -2.772170723004       -9.47       -5.07    1.0    112ms
  8   -2.772170723010      -11.21       -5.12    1.0    125ms
  9   -2.772170723014      -11.41       -5.64    1.0    116ms
 10   -2.772170723015      -12.30       -5.83    2.0    147ms
 11   -2.772170723015      -12.76       -6.04    1.0    126ms
 12   -2.772170723015      -13.20       -6.32    1.0    125ms
 13   -2.772170723015      -13.94       -6.64    1.0    133ms
 14   -2.772170723015   +  -15.35       -7.98    1.0    125ms
 15   -2.

1.7735579492840987

We do the same thing using automatic differentiation. Under the hood this uses
custom rules to implicitly differentiate through the self-consistent
field fixed-point problem.

In [3]:
polarizability = ForwardDiff.derivative(compute_dipole, 0.0)
println()
println("Polarizability via ForwardDiff:       $polarizability")
println("Polarizability via finite difference: $polarizability_fd")

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770740251334                   -0.52    9.0         
  2   -2.772059740510       -2.88       -1.32    1.0    103ms
  3   -2.772082965889       -4.63       -2.42    1.0    130ms
  4   -2.772083328926       -6.44       -3.11    1.0    126ms
  5   -2.772083417630       -7.05       -4.33    2.0    171ms
  6   -2.772083417738       -9.97       -4.63    1.0    109ms
  7   -2.772083417809      -10.15       -5.78    1.0    112ms
  8   -2.772083417811      -11.81       -6.47    2.0    137ms
  9   -2.772083417811   +  -13.97       -6.57    2.0    126ms
 10   -2.772083417811      -13.85       -8.23    1.0    129ms

Polarizability via ForwardDiff:       1.7725349591124775
Polarizability via finite difference: 1.7735579492840987
